In [ ]:
import requests
import json


url = 'https://raw.githubusercontent.com/openfootball/football.json/master/2015-16/en.1.json'


response = requests.get(url)

# Vérifier si la requête a réussi
if response.status_code == 200:
    # Charger les données JSON
    data = response.json()
    # Afficher les données
    print(json.dumps(data, indent=4))
else:
    print(f"Erreur lors du téléchargement du fichier: {response.status_code}")


{
    "name": "English Premier League 2015/16",
    "matches": [
        {
            "round": "Matchday 1",
            "date": "2015-08-08",
            "time": "12:45",
            "team1": "Manchester United",
            "team2": "Tottenham Hotspur",
            "score": {
                "ht": [
                    1,
                    0
                ],
                "ft": [
                    1,
                    0
                ]
            }
        },
        {
            "round": "Matchday 1",
            "date": "2015-08-08",
            "time": "15:00",
            "team1": "AFC Bournemouth",
            "team2": "Aston Villa",
            "score": {
                "ht": [
                    0,
                    0
                ],
                "ft": [
                    0,
                    1
                ]
            }
        },
        {
            "round": "Matchday 1",
            "date": "2015-08-08",
            "time": "15:00",
     

In [11]:
data.keys(), data["matches"]

(dict_keys(['name', 'matches']),
 [{'round': 'Matchday 1',
   'date': '2015-08-08',
   'time': '12:45',
   'team1': 'Manchester United',
   'team2': 'Tottenham Hotspur',
   'score': {'ht': [1, 0], 'ft': [1, 0]}},
  {'round': 'Matchday 1',
   'date': '2015-08-08',
   'time': '15:00',
   'team1': 'AFC Bournemouth',
   'team2': 'Aston Villa',
   'score': {'ht': [0, 0], 'ft': [0, 1]}},
  {'round': 'Matchday 1',
   'date': '2015-08-08',
   'time': '15:00',
   'team1': 'Everton FC',
   'team2': 'Watford FC',
   'score': {'ht': [0, 1], 'ft': [2, 2]}},
  {'round': 'Matchday 1',
   'date': '2015-08-08',
   'time': '15:00',
   'team1': 'Leicester City',
   'team2': 'Sunderland AFC',
   'score': {'ht': [3, 0], 'ft': [4, 2]}},
  {'round': 'Matchday 1',
   'date': '2015-08-08',
   'time': '15:00',
   'team1': 'Norwich City',
   'team2': 'Crystal Palace',
   'score': {'ht': [0, 1], 'ft': [1, 3]}},
  {'round': 'Matchday 1',
   'date': '2015-08-08',
   'time': '17:30',
   'team1': 'Chelsea FC',
   'te